In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('model-selection-clean.csv', index_col = 0)

In [3]:
df.head()

,SCHOOL,STATE,ID,ABBR,COUNTY,CHARTER,MAGNET,LOCALE,TITLE I,TITLE I ELIGIBLE,...,AMERICAN INDIAN,ASIAN,HISPANIC,BLACK,WHITE,TOTAL RACE/ETHINICITY,FTE TEACHERS,PUPIL-TEACHER RATIO,FREE LUNCH ELIGIBLE NUMERIC,REDUCED PRICE LUNCH ELIGIBLE NUMERIC
1,10TH STREET SCHOOL,Washington,5.304860e+11,WA,Snohomish County,2-No,–,22-Suburb: Mid-size,1-Yes,1-Yes,...,8,5,20,1,122,0,0,0,28.0,3.0
2,112 ALC INDEPENDENT STUDY,Minnesota,2.708190e+11,MN,Carver County,2-No,2-No,21-Suburb: Large,†,2-No,...,0,1,10,0,22,0,0,0,8.0,3.0
3,112 ALC MIDDLE SCHOOL,Minnesota,2.708190e+11,MN,Carver County,2-No,2-No,21-Suburb: Large,†,2-No,...,0,5,32,8,82,0,0,0,54.0,7.0
4,12TH STREET ELEMENTARY,Michigan,2.628950e+11,MI,Kalamazoo County,2-No,2-No,13-City: Small,†,2-No,...,0,68,27,48,406,0,0,0,99.0,21.0
6,1R ELEMENTARY,Nebraska,3.174580e+11,NE,Hall County,†,†,41-Rural: Fringe,†,2-No,...,1,0,12,3,155,0,13,0,15.0,30.0


In [4]:
df.columns

Index(['SCHOOL', 'STATE', 'ID', 'ABBR', 'COUNTY', 'CHARTER', 'MAGNET',
       'LOCALE', 'TITLE I', 'TITLE I ELIGIBLE', 'TOTAL STUDENTS',
       'FREE LUNCH ELIGIBLE', 'REDUCED PRICE LUNCH ELIGIBLE',
       'FREE AND REDUCED LUNCH ELIGIBLE', 'MALE STUDENTS', 'FEMALE STUDENTS',
       'AMERICAN INDIAN', 'ASIAN', 'HISPANIC', 'BLACK', 'WHITE',
       'TOTAL RACE/ETHINICITY', 'FTE TEACHERS', 'PUPIL-TEACHER RATIO',
       'FREE LUNCH ELIGIBLE NUMERIC', 'REDUCED PRICE LUNCH ELIGIBLE NUMERIC'],
      dtype='object')

In [5]:
df.shape

(338254, 26)

In [6]:
df.drop(columns = ['SCHOOL', 'STATE', 'ID', 'COUNTY', 'CHARTER', 'MAGNET', 'TITLE I',
                   'REDUCED PRICE LUNCH ELIGIBLE', 'FREE AND REDUCED LUNCH ELIGIBLE',
                  'MALE STUDENTS', 'FEMALE STUDENTS', 'TOTAL RACE/ETHINICITY', 'FTE TEACHERS', 'PUPIL-TEACHER RATIO'], inplace = True)

In [7]:
df.reset_index( drop = True, inplace = True)

Function to Drop invalid free lunch eligible students.

In [8]:
invalid_total_students = df['TOTAL STUDENTS'].apply( lambda x : x if ( (str(x)).isdigit() and x != 0) else None)

In [9]:
invalid_total_students = invalid_total_students[invalid_total_students.isna() == True].index

In [10]:
df.drop(invalid_total_students, inplace = True)

In [11]:
df.reset_index(inplace = True, drop = True)

In [12]:
invalid_idx = df['FREE LUNCH ELIGIBLE'].apply( lambda x : None if (not str(x).isdigit() and df['FREE LUNCH ELIGIBLE'] > df['TOTAL STUDENTS']) else x)

In [13]:
invalid_idx = invalid_idx[invalid_idx.isna() == True].index

In [14]:
df.drop(invalid_idx, inplace = True)

In [15]:
df.shape

(337803, 12)

# Adding Label

In [16]:
poverty = df['FREE LUNCH ELIGIBLE']/df['TOTAL STUDENTS']

In [17]:
hi_poverty = poverty.apply( lambda x : 1 if x > 0.233 else 0)

In [18]:
strict_poverty = poverty.apply( lambda x : 1 if x > 0.339 else 0)

In [19]:
hi_poverty.groupby(hi_poverty).count()

0     74769
1    263034
dtype: int64

In [20]:
strict_poverty.groupby(strict_poverty).count()

0    120118
1    217685
dtype: int64

In [21]:
df['HI-LABEL'] = hi_poverty
df['STRICT-LABEL'] = strict_poverty

In [22]:
df.head()

,ABBR,LOCALE,TITLE I ELIGIBLE,TOTAL STUDENTS,FREE LUNCH ELIGIBLE,AMERICAN INDIAN,ASIAN,HISPANIC,BLACK,WHITE,FREE LUNCH ELIGIBLE NUMERIC,REDUCED PRICE LUNCH ELIGIBLE NUMERIC,HI-LABEL,STRICT-LABEL
0,WA,22-Suburb: Mid-size,1-Yes,171,28,8,5,20,1,122,28.0,3.0,0,0
1,MN,21-Suburb: Large,2-No,34,8,0,1,10,0,22,8.0,3.0,1,0
2,MN,21-Suburb: Large,2-No,130,54,0,5,32,8,82,54.0,7.0,1,1
3,MI,13-City: Small,2-No,594,99,0,68,27,48,406,99.0,21.0,0,0
4,NE,41-Rural: Fringe,2-No,171,15,1,0,12,3,155,15.0,30.0,0,0


In [23]:
#df['HI-LABEL'].to_csv('High-label.csv')

In [24]:
#df['STRICT-LABEL'].to_csv('Strict-label.csv')

In [25]:
df.drop( columns = ['TITLE I ELIGIBLE', 'FREE LUNCH ELIGIBLE', 'FREE LUNCH ELIGIBLE NUMERIC', 'REDUCED PRICE LUNCH ELIGIBLE NUMERIC', 'HI-LABEL', 'STRICT-LABEL'], inplace = True)

In [26]:
df.head()

,ABBR,LOCALE,TOTAL STUDENTS,AMERICAN INDIAN,ASIAN,HISPANIC,BLACK,WHITE
0,WA,22-Suburb: Mid-size,171,8,5,20,1,122
1,MN,21-Suburb: Large,34,0,1,10,0,22
2,MN,21-Suburb: Large,130,0,5,32,8,82
3,MI,13-City: Small,594,0,68,27,48,406
4,NE,41-Rural: Fringe,171,1,0,12,3,155


In [27]:
df['ABBR'] = df['ABBR'].astype('str')

In [28]:
state_code = {
    'CA ':1,'WY ':2,'MO ':3,'TX ':4,'OK ':5,'MS ':6, 'KY ':7,'ID ':8,'OR ':9,'WI ':10, 'ME ':11,'MA ':12,'NC ':13,
    'VA ':14, 'OH ':15,'IN ':16, 'AL ':17, 'IA ':18, 'FL ':19, 'LA ':20, 'CT ':21, 'SC ':22, 'GA ':23, 'MD ':24,
    'NE ':25, 'SD ':26, 'MT ':27, 'RI ':28, 'UT ':29, 'WV ':30, 'AR ':31, 'NV ':32, 'HI ':33, 'DE ':34, 'NY ':35,
    'DE ':37, 'NY ':38, 'MN ':39, 'NM ':40, 'NJ ':41,'PA ':42, 'ND ':43, 'NH ': 44, 'MI ':45, 'KS ':46, 'CO ':47,
    'VT ':48, 'DC ':49, 'WA ':50, 'IL ':51, 'AK ': 52, 'AZ ':53,'TN ':54
}

In [29]:
print(state_code.keys())

dict_keys(['CA ', 'WY ', 'MO ', 'TX ', 'OK ', 'MS ', 'KY ', 'ID ', 'OR ', 'WI ', 'ME ', 'MA ', 'NC ', 'VA ', 'OH ', 'IN ', 'AL ', 'IA ', 'FL ', 'LA ', 'CT ', 'SC ', 'GA ', 'MD ', 'NE ', 'SD ', 'MT ', 'RI ', 'UT ', 'WV ', 'AR ', 'NV ', 'HI ', 'DE ', 'NY ', 'MN ', 'NM ', 'NJ ', 'PA ', 'ND ', 'NH ', 'MI ', 'KS ', 'CO ', 'VT ', 'DC ', 'WA ', 'IL ', 'AK ', 'AZ ', 'TN '])


In [30]:
print( df['ABBR'].unique())

['WA ' 'MN ' 'MI ' 'NE ' 'KY ' 'IN ' 'PA ' 'TX ' 'WI ' 'KS ' 'CA ' 'IL '
 'NY ' 'RI ' 'ID ' 'WY ' 'MO ' 'CO ' 'OK ' 'NM ' 'NJ ' 'OR ' 'NC ' 'VA '
 'OH ' 'AZ ' 'ND ' 'MS ' 'LA ' 'CT ' 'SC ' 'FL ' 'NH ' 'MD ' 'WV ' 'GA '
 'SD ' 'AK ' 'IA ' 'ME ' 'MT ' 'DE ' 'AR ' 'UT ' 'NV ' 'VT ' 'DC ' 'HI '
 'TN ' 'AL ']


In [31]:
locale_code = {
    '3-Urban fringe of large city': 101, '6-Small town': 102, '5-Large town': 103, '2-Midsize city':104, 
    '8-Rural - inside CBSA/MSA':105,'7-Rural - outside CBSA/MSA':106, '4-Urban fringe of midsize city':107,
    '1-Large city':108, '41-Rural: Fringe':109, '22-Suburb: Mid-size':110, '21-Suburb: Large':111, 
    '42-Rural: Distant':112, '11-City: Large':113, '13-City: Small':114, '43-Rural: Remote':115,
    '33-Town: Remote':116, '12-City: Mid-size':117, '32-Town: Distant':118, '31-Town: Fringe':119,
    '23-Suburb: Small':120
}

In [32]:
df['ABBR'] = df['ABBR'].apply(lambda x : state_code[x] if x in state_code else 0)
df['ABBR'] = df['ABBR'].astype('category')
df['ABBR'] = df['ABBR'].astype('str')

In [33]:
df['LOCALE'] = df['LOCALE'].apply(lambda x : locale_code[x] if x in locale_code else 121)
df['LOCALE'] = df['LOCALE'].astype('category')
df['LOCALE'] = df['LOCALE'].astype('str')

In [34]:
num_cat = ['TOTAL STUDENTS', 'ASIAN', 'AMERICAN INDIAN','HISPANIC', 'BLACK', 'WHITE']
scl = StandardScaler()

for col in num_cat:
    df[col] = scl.fit_transform(df[[col]].values)

In [35]:
df.head()

,ABBR,LOCALE,TOTAL STUDENTS,AMERICAN INDIAN,ASIAN,HISPANIC,BLACK,WHITE
0,50,110,-0.820101,0.118183,-0.280551,-0.521809,-0.520741,-0.490510
1,39,111,-1.122707,-0.217000,-0.327979,-0.561820,-0.527422,-0.854489
2,39,111,-0.910662,-0.217000,-0.280551,-0.473797,-0.473977,-0.636101
3,45,114,0.114222,-0.217000,0.466429,-0.493802,-0.206750,0.543192
4,25,109,-0.820101,-0.175102,-0.339835,-0.553817,-0.507380,-0.370396


In [36]:
#df.to_csv('data_for_model_selection.csv')